In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf

### 시퀀스를 예측하는 머신러닝 모델 만들기

#### 윈도우 데이터 셋 만들기

In [14]:
def trend(time, slope=0):
    return slope * time

In [15]:
def seasonal_pattern(season_time):
    return np.where(season_time < 0.4,
    np.cos(season_time * 2 * np.pi),
    1 / np.exp(3 * season_time))

In [16]:
def seasonality(time, period, amplitude=1, phase=0):
    season_time = ((time + phase) % period) / period
    return amplitude * seasonal_pattern(season_time)

In [17]:
def noise(time, noise_level=1, seed = None):
    rnd = np.random.RandomState(seed)
    return rnd.randn(len(time)) * noise_level

In [18]:
time = np.arange(4 * 365 + 1, dtype="float32")
baseline = 10
series = trend(time, .05)
amplitude = 15
slope = 0.09
noise_level = 6

In [20]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    dataset =  tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    # 반환 값이 dataset
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    # [0 1 2 3 4] 형식으로 바로 받기 위해서 flat_map함수를 사용한다.
    dataset = dataset.shuffle(shuffle_buffer).map(lambda window: (window[:-1], window[-1]))
    dataset = dataset.batch(batch_size).prefetch(1)
    # prefetch 훈련속도에 영향을 준다. 하나의 데이터셋이 작업을 하는 동안 미리 다른 데이터셋을 준비
    return dataset

In [19]:
series = baseline + trend(time, slope) \
                  + seasonality(time, period=365, amplitude=amplitude)

In [21]:
split_time = 1000
time_train = time[:split_time]
x_train = series[:split_time]
time_valid = time[split_time:]
x_valid = series[split_time:]
window_size = 20
batch_size = 32
shuffle_buffer_size = 1000
dataset = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)

In [22]:
# 데이터셋을 윈도우데이터셋으로 바꿔 주고 for문을 통해서 feature과 label로 나누어 준다.
dataset = windowed_dataset(series, window_size, 1, shuffle_buffer_size)
for feature, label in dataset.take(1):
    print(feature)
    print(label)

tf.Tensor(
[[55.530388 55.391083 55.24977  55.106514 54.961388 54.814457 54.6658
  54.515476 54.363564 54.210133 54.055256 53.899006 53.741455 53.58268
  53.42275  53.261738 53.099724 52.936775 52.772976 52.608395]], shape=(1, 20), dtype=float32)
tf.Tensor([52.443104], shape=(1,), dtype=float32)


#### DNN

In [23]:
dataset = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10, input_shape=[window_size],
                        activation="relu"),
    tf.keras.layers.Dense(10, activation="relu"),
    tf.keras.layers.Dense(1)
])

##### SGD를 사용하는 이유는 전체 학습데이터에서 하나씩 랜덤으로 뽑아 학습을 진행하기 때문에 모든 데이터를 사용하지 않아 속도가 빠르다는 장점이 있습니다. 하지만 지그재그로 핑퐁하면서 불안정한 학습과정을 보입니다. 단점으로는 지역 최소점과 안장점에 빠질 수 있습니다.

In [24]:
sgd = tf.keras.optimizers.SGD(learning_rate=1e-6, momentum=0.9)
model.compile(loss="mse", optimizer=sgd)

In [25]:
model.fit(dataset,epochs=100,verbose=1)

Epoch 1/100
31/31 [==============================] - 4s 89ms/step - loss: 28.4238
Epoch 2/100
31/31 [==============================] - 0s 2ms/step - loss: 20.7115
Epoch 3/100
31/31 [==============================] - 0s 2ms/step - loss: 18.7199
Epoch 4/100
31/31 [==============================] - 0s 2ms/step - loss: 17.7870
Epoch 5/100
31/31 [==============================] - 0s 2ms/step - loss: 17.3353
Epoch 6/100
31/31 [==============================] - 0s 3ms/step - loss: 16.1136
Epoch 7/100
31/31 [==============================] - 0s 2ms/step - loss: 15.5023
Epoch 8/100
31/31 [==============================] - 0s 2ms/step - loss: 15.0602
Epoch 9/100
31/31 [==============================] - 0s 2ms/step - loss: 14.4923
Epoch 10/100
31/31 [==============================] - 0s 2ms/step - loss: 14.5317
Epoch 11/100
31/31 [==============================] - 0s 2ms/step - loss: 13.1581
Epoch 12/100
31/31 [==============================] - 0s 2ms/step - loss: 12.7790
Epoch 13/100
31/31 [====

### DNN 결과 평가하기

In [26]:
print(series[1000:1020])

[101.63048  101.70713  101.78389  101.86077  101.937744 102.01483
 102.09202  102.16932  102.24672  102.32423  102.401825 102.47954
 102.55734  102.635254 102.71326  102.79136  102.86956  102.94786
 103.02625  103.104744]


In [27]:
print(series[1020])

103.18333


In [28]:
print(model.predict(series[1000:1020][np.newaxis]))

1/1 [==============================] - 0s 198ms/step
[[103.262]]
